# NETFLIX SUBSCRIPTION PROBLEM

## LIBRARIES AND NECESSARY UTILITIES 

In [1]:
#Import some libraries
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sb
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    import matplotlib.cm as cm
    from sklearn.decomposition import PCA

pd.options.display.max_rows = 2200
pd.options.display.max_columns = 2200

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier as cat
from sklearn.metrics import roc_auc_score,log_loss
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

C:\Users\DEV\Anaconda3\lib\site-packages\distributed\utils.py:133: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [WinError 10065] A socket operation was attempted to an unreachable host
  RuntimeWarning,


## LOADING THE DATASETS 

In [2]:
train = pd.read_csv('train.csv')
test=  pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [3]:
train.head(2)

,id,target,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_83,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_100,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_149,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_187,feature_188,feature_189,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199,feature_200,feature_201,feature_202,feature_203,feature_204,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_227,feature_228,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_248,feature_249,feature_250,feature_251,feature_252,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_312,feature_313,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature

In [4]:
train.shape

(70000, 509)

In [5]:
#view the data types in the data data
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 509 entries, id to feature_506
dtypes: float64(143), int64(274), object(92)
memory usage: 271.8+ MB


In [6]:
duplicate = train[train.duplicated()]
duplicate.shape

(0, 509)

In [7]:
# check unique values in dataset
train.apply(lambda x: len(x.unique()))

id             70000
target             2
feature_0          2
feature_1          2
feature_2          6
feature_3          6
feature_4         12
feature_5       8489
feature_6      14005
feature_7      18387
feature_8       1708
feature_9       3940
feature_10      7122
feature_11       354
feature_12       888
feature_13      1643
feature_14      3439
feature_15      8157
feature_16         4
feature_17       103
feature_18     15858
feature_19       260
feature_20      5841
feature_21      2327
feature_22      2339
feature_23         2
feature_24        25
feature_25     21148
feature_26     36252
feature_27         2
feature_28         2
feature_29         2
feature_30         2
feature_31         2
feature_32         2
feature_33         2
feature_34         2
feature_35     24699
feature_36     26668
feature_37     65856
feature_38     64115
feature_39     24672
feature_40     26593
feature_41         2
feature_42         2
feature_43         3
feature_44         3
feature_45   

Drop columns with only one unique category

In [8]:
# droping single catgorical features
single_feature_columns = ['feature_190', 'feature_55', 'feature_187', 'feature_188', 'feature_249', 'feature_248','feature_461', 'feature_474', 'feature_472', 'feature_409',
       'feature_469', 'feature_467', 'feature_411', 'feature_465',
       'feature_312', 'feature_463', 'feature_313', 'feature_375',
       'feature_427', 'feature_412', 'feature_413', 'feature_455',
       'feature_453', 'feature_419', 'feature_451', 'feature_100',
       'feature_449', 'feature_421', 'feature_445', 'feature_433',
       'feature_432', 'feature_459', 'feature_476', 'feature_405',
       'feature_425', 'feature_498', 'feature_227', 'feature_228',
       'feature_149', 'feature_495', 'feature_385', 'feature_493',
       'feature_478', 'feature_491', 'feature_389', 'feature_489',
       'feature_252', 'feature_487', 'feature_502', 'feature_505',
       'feature_484', 'feature_395', 'feature_482', 'feature_377',
       'feature_358', 'feature_480', 'feature_397', 'feature_357',
       'feature_500']

train = train.drop(columns=single_feature_columns)
test = test.drop(columns=single_feature_columns)
train.shape, test.shape

((70000, 450), (30000, 449))

In [9]:
# #check for missing values
# percent_missing = train.isnull().sum() * 100 / len(train)
# missing_value_train = pd.DataFrame({'column_name': train.columns,
#                                  'percent_missing': percent_missing})
# missing_value_train

Drop the columns with 100% missing values

In [10]:
# Drop those all null value features
null_columns = ['feature_193', 'feature_196', 'feature_197', 'feature_198', 'feature_83']
train = train.drop(columns=null_columns)
test = test.drop(columns=null_columns)

train.shape, test.shape

((70000, 445), (30000, 444))

Extracting the other features like year,day,month end weekday,start weekday and quarter from the date columns in the dataset

In [11]:
date_columns = ['feature_191', 'feature_192', 'feature_194', 'feature_195', 'feature_199', 'feature_200', 'feature_201', 'feature_202', 'feature_203', 'feature_204']

In [12]:
train[date_columns].apply(lambda x: len(x.unique()))

feature_191    6731
feature_192    2921
feature_194    3180
feature_195    2739
feature_199    2479
feature_200    3129
feature_201    3265
feature_202    2724
feature_203    2858
feature_204    3574
dtype: int64

In [13]:
for c in date_columns:
    train[c] = pd.to_datetime(train[c])
    test[c] = pd.to_datetime(test[c])
    # extract new ceatures crom date
    train[c + '_day'] = train[c].dt.day
    test[c + '_day'] = test[c].dt.day    
    train[c + '_year'] = train[c].dt.year
    test[c + '_year'] = test[c].dt.year
    train[c + '_month'] = train[c].dt.month
    test[c + '_month'] = test[c].dt.month
    train[c + '_weekday'] = train[c].dt.weekday
    test[c + '_weekday'] = test[c].dt.weekday
    train[c + '_quarter'] = train[c].dt.quarter
    test[c + '_quarter'] = test[c].dt.quarter

    # remove original data column
    train = train.drop(c, axis=1)
    test = test.drop(c, axis=1)

In [14]:
new_date_columns = ['feature_191_day','feature_191_year','feature_191_month','feature_191_weekday','feature_191_quarter',
                'feature_192_day','feature_192_year','feature_192_month','feature_192_weekday','feature_192_quarter', 
                'feature_194_day','feature_194_year','feature_194_month','feature_194_weekday','feature_194_quarter',
                'feature_195_day','feature_195_year','feature_195_month','feature_195_weekday','feature_195_quarter', 
                'feature_199_day', 'feature_199_year','feature_199_month','feature_199_weekday','feature_199_quarter', 
                'feature_200_day','feature_200_year','feature_200_month','feature_200_weekday','feature_200_quarter',
                'feature_201_day','feature_201_year','feature_201_month','feature_201_weekday','feature_201_quarter',
                'feature_202_day','feature_202_year','feature_202_month','feature_202_weekday','feature_202_quarter',
                'feature_203_day','feature_203_year','feature_203_month','feature_203_weekday','feature_203_quarter',
                'feature_204_day','feature_204_year','feature_204_month','feature_204_weekday','feature_204_quarter']

In [18]:
train[new_date_columns] = train[new_date_columns].astype('category')
test[new_date_columns] = test[new_date_columns].astype('category')

In [16]:
train['feature_191_day'] = train['feature_191_day'].cat.as_ordered()
train['feature_191_year'] = train['feature_191_year'].cat.as_ordered()
train['feature_191_month'] = train['feature_191_month'].cat.as_ordered()
train['feature_191_weekday'] = train['feature_191_weekday'].cat.as_ordered()
train['feature_191_quarter'] = train['feature_191_quarter'].cat.as_ordered()

train['feature_192_day'] = train['feature_192_day'].cat.as_ordered()
train['feature_192_year'] = train['feature_192_year'].cat.as_ordered()
train['feature_192_month'] = train['feature_192_month'].cat.as_ordered()
train['feature_192_weekday'] = train['feature_192_weekday'].cat.as_ordered()
train['feature_192_quarter'] = train['feature_192_quarter'].cat.as_ordered()

train['feature_194_day'] = train['feature_194_day'].cat.as_ordered()
train['feature_194_year'] = train['feature_194_year'].cat.as_ordered()
train['feature_194_month'] = train['feature_194_month'].cat.as_ordered()
train['feature_194_weekday'] = train['feature_194_weekday'].cat.as_ordered()
train['feature_194_quarter'] = train['feature_194_quarter'].cat.as_ordered()

train['feature_195_day'] = train['feature_195_day'].cat.as_ordered()
train['feature_195_year'] = train['feature_195_year'].cat.as_ordered()
train['feature_195_month'] = train['feature_195_month'].cat.as_ordered()
train['feature_195_weekday'] = train['feature_195_weekday'].cat.as_ordered()
train['feature_195_quarter'] = train['feature_195_quarter'].cat.as_ordered()

train['feature_199_day'] = train['feature_199_day'].cat.as_ordered()
train['feature_199_year'] = train['feature_199_year'].cat.as_ordered()
train['feature_199_month'] = train['feature_199_month'].cat.as_ordered()
train['feature_199_weekday'] = train['feature_199_weekday'].cat.as_ordered()
train['feature_199_quarter'] = train['feature_199_quarter'].cat.as_ordered()

train['feature_200_day'] = train['feature_200_day'].cat.as_ordered()
train['feature_200_year'] = train['feature_200_year'].cat.as_ordered()
train['feature_200_month'] = train['feature_200_month'].cat.as_ordered()
train['feature_200_weekday'] = train['feature_200_weekday'].cat.as_ordered()
train['feature_200_quarter'] = train['feature_200_quarter'].cat.as_ordered()

train['feature_201_day'] = train['feature_201_day'].cat.as_ordered()
train['feature_201_year'] = train['feature_201_year'].cat.as_ordered()
train['feature_201_month'] = train['feature_201_month'].cat.as_ordered()
train['feature_201_weekday'] = train['feature_201_weekday'].cat.as_ordered()
train['feature_201_quarter'] = train['feature_201_quarter'].cat.as_ordered()

train['feature_202_day'] = train['feature_202_day'].cat.as_ordered()
train['feature_202_year'] = train['feature_202_year'].cat.as_ordered()
train['feature_202_month'] = train['feature_202_month'].cat.as_ordered()
train['feature_202_weekday'] = train['feature_202_weekday'].cat.as_ordered()
train['feature_202_quarter'] = train['feature_202_quarter'].cat.as_ordered()

train['feature_203_day'] = train['feature_203_day'].cat.as_ordered()
train['feature_203_year'] = train['feature_203_year'].cat.as_ordered()
train['feature_203_month'] = train['feature_203_month'].cat.as_ordered()
train['feature_203_weekday'] = train['feature_203_weekday'].cat.as_ordered()
train['feature_203_quarter'] = train['feature_203_quarter'].cat.as_ordered()

train['feature_204_day'] = train['feature_204_day'].cat.as_ordered()
train['feature_204_year'] = train['feature_204_year'].cat.as_ordered()
train['feature_204_month'] = train['feature_204_month'].cat.as_ordered()
train['feature_204_weekday'] = train['feature_204_weekday'].cat.as_ordered()
train['feature_204_quarter'] = train['feature_204_quarter'].cat.as_ordered()





In [19]:
test['feature_191_day'] = test['feature_191_day'].cat.as_ordered()
test['feature_191_year'] = test['feature_191_year'].cat.as_ordered()
test['feature_191_month'] = test['feature_191_month'].cat.as_ordered()
test['feature_191_weekday'] = test['feature_191_weekday'].cat.as_ordered()
test['feature_191_quarter'] = test['feature_191_quarter'].cat.as_ordered()

test['feature_192_day'] = test['feature_192_day'].cat.as_ordered()
test['feature_192_year'] = test['feature_192_year'].cat.as_ordered()
test['feature_192_month'] = test['feature_192_month'].cat.as_ordered()
test['feature_192_weekday'] = test['feature_192_weekday'].cat.as_ordered()
test['feature_192_quarter'] = test['feature_192_quarter'].cat.as_ordered()

test['feature_194_day'] = test['feature_194_day'].cat.as_ordered()
test['feature_194_year'] = test['feature_194_year'].cat.as_ordered()
test['feature_194_month'] = test['feature_194_month'].cat.as_ordered()
test['feature_194_weekday'] = test['feature_194_weekday'].cat.as_ordered()
test['feature_194_quarter'] = test['feature_194_quarter'].cat.as_ordered()

test['feature_195_day'] = test['feature_195_day'].cat.as_ordered()
test['feature_195_year'] = test['feature_195_year'].cat.as_ordered()
test['feature_195_month'] = test['feature_195_month'].cat.as_ordered()
test['feature_195_weekday'] = test['feature_195_weekday'].cat.as_ordered()
test['feature_195_quarter'] = test['feature_195_quarter'].cat.as_ordered()

test['feature_199_day'] = test['feature_199_day'].cat.as_ordered()
test['feature_199_year'] = test['feature_199_year'].cat.as_ordered()
test['feature_199_month'] = test['feature_199_month'].cat.as_ordered()
test['feature_199_weekday'] = test['feature_199_weekday'].cat.as_ordered()
test['feature_199_quarter'] = test['feature_199_quarter'].cat.as_ordered()

test['feature_200_day'] = test['feature_200_day'].cat.as_ordered()
test['feature_200_year'] = test['feature_200_year'].cat.as_ordered()
test['feature_200_month'] = test['feature_200_month'].cat.as_ordered()
test['feature_200_weekday'] = test['feature_200_weekday'].cat.as_ordered()
test['feature_200_quarter'] = test['feature_200_quarter'].cat.as_ordered()

test['feature_201_day'] = test['feature_201_day'].cat.as_ordered()
test['feature_201_year'] = test['feature_201_year'].cat.as_ordered()
test['feature_201_month'] = test['feature_201_month'].cat.as_ordered()
test['feature_201_weekday'] = test['feature_201_weekday'].cat.as_ordered()
test['feature_201_quarter'] = test['feature_201_quarter'].cat.as_ordered()

test['feature_202_day'] = test['feature_202_day'].cat.as_ordered()
test['feature_202_year'] = test['feature_202_year'].cat.as_ordered()
test['feature_202_month'] = test['feature_202_month'].cat.as_ordered()
test['feature_202_weekday'] = test['feature_202_weekday'].cat.as_ordered()
test['feature_202_quarter'] = test['feature_202_quarter'].cat.as_ordered()

test['feature_203_day'] = test['feature_203_day'].cat.as_ordered()
test['feature_203_year'] = test['feature_203_year'].cat.as_ordered()
test['feature_203_month'] = test['feature_203_month'].cat.as_ordered()
test['feature_203_weekday'] = test['feature_203_weekday'].cat.as_ordered()
test['feature_203_quarter'] = test['feature_203_quarter'].cat.as_ordered()

test['feature_204_day'] = test['feature_204_day'].cat.as_ordered()
test['feature_204_year'] = test['feature_204_year'].cat.as_ordered()
test['feature_204_month'] = test['feature_204_month'].cat.as_ordered()
test['feature_204_weekday'] = test['feature_204_weekday'].cat.as_ordered()
test['feature_204_quarter'] = test['feature_204_quarter'].cat.as_ordered()





In [20]:
# train[new_date_columns].apply(lambda x: len(x.unique()))
train['feature_195_year'].isnull().sum()

44723

In [21]:
# # Refrence (edited): https://datavizpyr.com/visualizing-missing-data-with-seaborn-heatmap-and-displot/
# plt.figure(figsize=(18,16))
# sb.displot(
#     data=train[new_date_columns].isna().melt(value_name="missing"),
#     y="variable",
#     hue="missing",
#     multiple="fill",
#     aspect=3,
#     palette='copper' 
# )
# plt.title('Bar plot showing Non-Missing Values in Train data', weight = 'bold', size = 20, color = 'brown')
# plt.xlabel(" ")
# plt.ylabel(" ")
# plt.xticks(size = 12, weight = 'bold', color = 'maroon')
# plt.yticks(size = 12, weight = 'bold', color = 'maroon');

# plt.figure(figsize=(18,10))
# sb.heatmap(train[new_date_columns].isna().transpose(),
#             cmap="copper",
#             cbar_kws={'label': 'Missing Data'})
# plt.title('Heatmap showing Missing Values in Train data', weight = 'bold', size = 20, color = 'brown')
# plt.xticks(size = 12, color = 'maroon')
# plt.yticks(size = 12, color = 'maroon')
# plt.show();

In [22]:
train.shape, test.shape

((70000, 485), (30000, 484))

In [23]:
cat_cols = [col for col in train.select_dtypes('object').columns]

for col in train.columns:
  if col in cat_cols:
    train[col] = train[col].astype('category')


In [24]:
cat_cols = [col for col in test.select_dtypes('object').columns]

for col in test.columns:
  if col in cat_cols:
    test[col] = test[col].astype('category')

In [25]:
# # !rm -r kuma_utils
# !git clone https://github.com/analokmaus/kuma_utils.git

In [26]:
# import sys
# sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

In [27]:
lgbmimpute = train.drop(['target'], axis=1)
fea = lgbmimpute.columns

In [28]:
%%time
lgbm_imtr = LGBMImputer(n_iter=500)

train_iterimp = lgbm_imtr.fit_transform(train[fea])
test_iterimp = lgbm_imtr.transform(test[fea])

# Create train test imputed dataframe
train_imp_df = pd.DataFrame(train_iterimp, columns=fea)
test_imp_df = pd.DataFrame(test_iterimp, columns=fea)

C:\Users\DEV\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\DEV\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Wall time: 37min 14s


In [55]:
#adding the Target column from train data
train_imp_df['target'] = train['target']
train_imp_df['id'] = train['id']
test_imp_df['id'] = test['id']

In [59]:
test_imp_df.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_189,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_250,feature_251,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,feature_342,feature_343,feature_344,feature_345,feature_346,feature_347,feature_348,feature_349,feature_350,feature_351,feature_352,feature_353,feature_354,feature_355,feature_356,feature_359,feature_360,feature_361,feature_362,feature_363,feature_364,feature_365,feature_366,feature_367,feature_368,feature_369,feature_370,feature_371,

In [56]:
train_imp_df.shape,train.shape,test_imp_df.shape

((70000, 485), (70000, 485), (30000, 484))

In [61]:
train_imp_df.to_csv("lgbmimputetrain.csv",index=False)
test_imp_df.to_csv("lgbmimputetest.csv",index=False)

In [32]:
train_imp_df.apply(lambda x: len(x.unique()))

id                     70000
feature_0                  2
feature_1                  2
feature_2                  6
feature_3                  6
feature_4                 12
feature_5              53053
feature_6              57612
feature_7              60575
feature_8              46282
feature_9              47549
feature_10             49327
feature_11             44919
feature_12             44468
feature_13             43848
feature_14              3439
feature_15              8157
feature_16                 4
feature_17               103
feature_18             15858
feature_19               260
feature_20              5841
feature_21              2327
feature_22              2339
feature_23                 2
feature_24                25
feature_25             21148
feature_26             36252
feature_27                 2
feature_28                 2
feature_29                 2
feature_30                 2
feature_31                 2
feature_32                 2
feature_33    

In [33]:
# train[date_columns] = pd.to_datetime(train[date_columns],format="%d-%m-%Y",errors='coerce')

## LABEL ENCODING 

In [34]:
cat_columns = [col for col in train_imp_df.select_dtypes('category').columns]

In [35]:
# # label encoding those cat columns
def label_encoding(train, test, columns):
    for col in columns:
        le = LabelEncoder()
        values = train[col].append(test[col])
        le.fit(values)
        train[col] = le.transform(train[col])
        test[col] = le.transform(test[col])
    return train, test

train, test = label_encoding(train_imp_df, test_imp_df, cat_columns)
train[cat_columns].head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_41,feature_42,feature_54,feature_58,feature_97,feature_133,feature_134,feature_135,feature_152,feature_163,feature_164,feature_165,feature_185,feature_186,feature_189,feature_205,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_246,feature_247,feature_250,feature_251,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_336,feature_337,feature_338,feature_371,feature_199_day,feature_199_year,feature_199_month,feature_199_weekday,feature_199_quarter
0,0,0,1,5,3,0,61,5737,263,2808,0,0,0,11,0,0,0,0,0,0,0,0,0,0,248,3,28,2732,2,7,0,1,1,4,118,10,12,5,0,1,1,1,0,2,34,82,1,0,0,59,56,16,2,2,1,0,0,7,98,21,5,2,1,0,2,56,28,34,14,2,6,5,0,0,0,0,9,9,10,4,3
1,0,0,3,5,1,2,50,3082,263,1951,0,0,0,16,0,0,0,0,0,0,0,0,0,0,248,6,28,4915,2,7,0,3,1,3,118,12,13,9,0,2,2,1,6,3,79,60,1,0,0,74,67,20,5,5,2,1,0,27,6,26,9,6,3,2,0,31,7,26,9,7,3,2,0,0,0,0,3,9,1,5,0
2,0,0,3,5,4,1,78,9673,263,4066,0,0,0,15,0,0,0,0,0,0,0,0,0,0,248,6,28,660,2,7,0,3,1,3,118,8,3,9,0,2,2,1,6,3,63,60,1,0,0,74,67,20,5,5,2,1,0,27,6,26,9,6,3,2,0,31,7,26,9,7,3,2,0,0,0,0,23,8,2,3,0
3,0,0,1,5,1,0,58,5079,263,2516,0,0,0,12,0,0,0,0,0,0,0,0,0,0,248,6,28,5378,2,7,0,3,1,1,118,6,16,0,0,2,2,1,0,3,79,12,1,0,0,71,66,19,4,3,1,0,0,5,96,18,5,2,1,0,0,7,101,19,5,3,1,0,0,0,0,0,3,6,0,5,0
4,0,0,3,3,3,2,47,3016,263,1881,0,0,0,19,1,1,1,1,1,1,1,0,0,0,248,4,0,2915,2,7,0,0,1,3,118,4,3,2,0,1,1,1,0,2,117,29,1,0,0,57,54,14,2,2,1,0,0,123,79,15,3,2,1,0,0,18,112,16,3,3,1,0,0,0,0,1,8,5,9,2,3


In [36]:
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,feature_79,feature_80,feature_81,feature_82,feature_84,feature_85,feature_86,feature_87,feature_88,feature_89,feature_90,feature_91,feature_92,feature_93,feature_94,feature_95,feature_96,feature_97,feature_98,feature_99,feature_101,feature_102,feature_103,feature_104,feature_105,feature_106,feature_107,feature_108,feature_109,feature_110,feature_111,feature_112,feature_113,feature_114,feature_115,feature_116,feature_117,feature_118,feature_119,feature_120,feature_121,feature_122,feature_123,feature_124,feature_125,feature_126,feature_127,feature_128,feature_129,feature_130,feature_131,feature_132,feature_133,feature_134,feature_135,feature_136,feature_137,feature_138,feature_139,feature_140,feature_141,feature_142,feature_143,feature_144,feature_145,feature_146,feature_147,feature_148,feature_150,feature_151,feature_152,feature_153,feature_154,feature_155,feature_156,feature_157,feature_158,feature_159,feature_160,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170,feature_171,feature_172,feature_173,feature_174,feature_175,feature_176,feature_177,feature_178,feature_179,feature_180,feature_181,feature_182,feature_183,feature_184,feature_185,feature_186,feature_189,feature_205,feature_206,feature_207,feature_208,feature_209,feature_210,feature_211,feature_212,feature_213,feature_214,feature_215,feature_216,feature_217,feature_218,feature_219,feature_220,feature_221,feature_222,feature_223,feature_224,feature_225,feature_226,feature_229,feature_230,feature_231,feature_232,feature_233,feature_234,feature_235,feature_236,feature_237,feature_238,feature_239,feature_240,feature_241,feature_242,feature_243,feature_244,feature_245,feature_246,feature_247,feature_250,feature_251,feature_253,feature_254,feature_255,feature_256,feature_257,feature_258,feature_259,feature_260,feature_261,feature_262,feature_263,feature_264,feature_265,feature_266,feature_267,feature_268,feature_269,feature_270,feature_271,feature_272,feature_273,feature_274,feature_275,feature_276,feature_277,feature_278,feature_279,feature_280,feature_281,feature_282,feature_283,feature_284,feature_285,feature_286,feature_287,feature_288,feature_289,feature_290,feature_291,feature_292,feature_293,feature_294,feature_295,feature_296,feature_297,feature_298,feature_299,feature_300,feature_301,feature_302,feature_303,feature_304,feature_305,feature_306,feature_307,feature_308,feature_309,feature_310,feature_311,feature_314,feature_315,feature_316,feature_317,feature_318,feature_319,feature_320,feature_321,feature_322,feature_323,feature_324,feature_325,feature_326,feature_327,feature_328,feature_329,feature_330,feature_331,feature_332,feature_333,feature_334,feature_335,feature_336,feature_337,feature_338,feature_339,feature_340,feature_341,feature_342,feature_343,feature_344,feature_345,feature_346,feature_347,feature_348,feature_349,feature_350,feature_351,feature_352,feature_353,feature_354,feature_355,feature_356,feature_359,feature_360,feature_361,feature_362,feature_363,feature_364,feature_365,feature_366,feature_367,feature_368,feature_369,feature_370,feature_371,

## PCA 

In [37]:
# from sklearn.decomposition import PCA

# pca = PCA(random_state=random_state).fit(X_proc)

# X_pca = pd.DataFrame(pca.transform(X_clean), index=X_proc.index)

# pca_comp = pd.DataFrame(pca.components_, index=X.columns)
# pca_comp.style.background_gradient(
#     vmin=-1, vmax=1, cmap=sns.color_palette("vlag", as_cmap=True))

In [38]:
# print('\nLoading files ...')
# X = train.drop(['target','id'], axis=1).values
# y = train['target'].values.astype(np.int8)
# target_names = np.unique(y)
# print('\nThere are %d unique target valuess in this dataset:' % (len(target_names)), target_names)

In [39]:
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

In [40]:
# scaled_columns=  train.drop(['target','id'], axis=1).columns

In [41]:
# scaled_columns

In [42]:
# x=pd.DataFrame(X_scaled,columns=scaled_columns)

In [43]:
# x.head()

In [44]:
# n_comp = 30
# # PCA
# print('\nRunning PCA ...')
# pca = PCA(n_components=n_comp, svd_solver='full', random_state=1001)
# X_pca = pca.fit_transform(x)
# print('Explained variance: %.4f' % pca.explained_variance_ratio_.sum())

# print('Individual variance contributions:')
# for j in range(n_comp):
#     print(pca.explained_variance_ratio_[j])

In [45]:
# colors = ['blue', 'red']
# plt.figure(1, figsize=(10, 10))

# for color, i, target_name in zip(colors, [0, 1], target_names):
#     plt.scatter(X_pca[y == i, 0], X_pca[y == i, 1], color=color, s=1,
#                 alpha=.8, label=target_name, marker='.')
# plt.legend(loc='best', shadow=False, scatterpoints=3)
# plt.title(
#         "Scatter plot of the training data projected on the 1st "
#         "and 2nd principal components")
# plt.xlabel("Principal axis 1 - Explains %.1f %% of the variance" % (
#         pca.explained_variance_ratio_[0] * 100.0))
# plt.ylabel("Principal axis 2 - Explains %.1f %% of the variance" % (
#         pca.explained_variance_ratio_[1] * 100.0))

# plt.savefig('pca-porto-01.png', dpi=150)
# plt.show()


## FEATURE SELECTION 

In [46]:
# Select main columns to be used in training
main_cols = train.columns.difference(['id','target'])
X = train[main_cols]
y = train.target
# Create the model with several hyperparameters
lgbm_params = {
    'n_estimators':1000,
#     'objective': 'classification',
    'metric': 'auc',
    'early_stopping_rounds':200,
    'verbose': -1,
    'boost_from_average': False,
    'min_data': 1,
    'num_leaves': 2,
    'learning_rate': 1,
    'min_data_in_bin': 1,
    'use_missing': True,
    'random_state':42,
#     'zero_as_missing': True
}
model = lgb.LGBMClassifier(**lgbm_params)
# model = cat(n_estimators=1000, max_depth=6, eval_metric='AUC', reg_lambda = 370,random_state=42)
#
# Split into training and validation set
train_features, valid_features, train_y, valid_y = train_test_split(X, y, test_size = 0.2, random_state = 42,stratify=y)
# Train using early stopping
model.fit(train_features, train_y,eval_set=[(valid_features, valid_y)])

[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=20, will be overridden by min_data=1. Current value: min_data_in_leaf=1
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[1]	valid_0's auc: 0.642752
[2]	valid_0's auc: 0.713274
[3]	valid_0's auc: 0.736709
[4]	valid_0's auc: 0.74043
[5]	valid_0's auc: 0.747592
[6]	valid_0's auc: 0.753687
[7]	valid_0's auc: 0.756476
[8]	valid_0's auc: 0.758104
[9]	valid_0's auc: 0.758386
[10]	valid_0's auc: 0.763003
[11]	valid_0's auc: 0.765209
[12]	valid_0's auc: 0.766754
[13]	valid_0's auc: 0.767759
[14]	valid_0's auc: 0.769263
[15]	valid_0's auc: 0.770306
[16]	valid_0's auc: 0.770743
[17]	valid_0's auc: 0.772663
[18]	valid_0's auc: 0.772627
[19]	valid_0's auc: 0.77311
[20]	valid_0's auc: 0.773319
[21]	valid_0's auc: 0.773183
[22]	valid_0's auc: 0.774108
[23]	valid_0's auc: 0.774226
[24]	valid_0's auc: 0.775133
[25]	valid_0's auc: 0.775169
[26]	vali

LGBMClassifier(boost_from_average=False, early_stopping_rounds=200,
               learning_rate=1, metric='auc', min_data=1, min_data_in_bin=1,
               n_estimators=1000, num_leaves=2, random_state=42,
               use_missing=True, verbose=-1)

In [47]:
pd.options.display.max_rows = 2200
#define a function to create variable importance dataframe
def get_lgbm_varimp(model, train_columns, max_vars=600):
    
    # Scikit-learn API LGBMClassifier or LGBMRegressor was fitted, 
    # so using feature_importances_ property
    feature_importances_perc = (model.feature_importances_ / sum(model.feature_importances_)) * 100
    cv_varimp_df = pd.DataFrame([train_columns, feature_importances_perc]).T

    cv_varimp_df.columns = ['feature_name', 'varimp (%)']

    cv_varimp_df.sort_values(by='varimp (%)', ascending=False, inplace=True)

    cv_varimp_df = cv_varimp_df.iloc[0:max_vars]   

    return cv_varimp_df
#
get_lgbm_varimp(model, X.columns)

,feature_name,varimp (%)
123,feature_2,1.79104
96,feature_189,1.79104
102,feature_191_year,1.49254
464,feature_81,1.49254
151,feature_206,1.49254
65,feature_159,1.49254
398,feature_457,1.49254
306,feature_352,1.49254
77,feature_17,1.49254
75,feature_168,1.19403


In [48]:
important_features= [
"feature_2",
"feature_189",
"feature_191_year",
"feature_81",
"feature_206",
"feature_159",
"feature_457",
"feature_352",
"feature_17",
"feature_168",
"feature_7",
"feature_307",
"feature_186",
"feature_297",
"feature_92",
"feature_169",
"feature_177",
"feature_350",
"feature_164",
"feature_13",
"feature_247",
"feature_222",
"feature_214",
"feature_170",
"feature_89",
"feature_264",
"feature_387",
"feature_11",
"feature_238",
"feature_201_year",
"feature_153",
"feature_10",
"feature_223",
"feature_160",
"feature_60",
"feature_3",
"feature_232",
"feature_155",
"feature_356",
"feature_220",
"feature_218",
"feature_216",
"feature_57",
"feature_237",
"feature_54",
"feature_208",
"feature_51",
"feature_192_day",
"feature_203_year",
"feature_5",
"feature_279",
"feature_38",
"feature_328",
"feature_201_day",
"feature_343",
"feature_341",
"feature_259",
"feature_91",
"feature_417",
"feature_348",
"feature_360",
"feature_133",
"feature_281",
"feature_347",
"feature_94",
"feature_8",
"feature_258",
"feature_67",
"feature_415",
"feature_255",
"feature_96",
"feature_40",
'feature_4',
"feature_262",
"feature_26",
"feature_97",
"feature_86",
"feature_344",
"feature_68",
"feature_263",
"feature_42",
"feature_205",
"feature_236",
"feature_314",
"feature_280",
"feature_310",
"feature_207",
"feature_203_day",
"feature_273",
"feature_381",
"feature_380",
"feature_379",
"feature_275",
"feature_201_weekday",
"feature_378",
"feature_320",
"feature_414",
"feature_287",
"feature_302",
"feature_260",
"feature_306",
"feature_441",
"feature_399",
"feature_322",
"feature_298",
"feature_243",
"feature_231",
"feature_230",
"feature_229",
"feature_226",
"feature_401",
"feature_225",
"feature_289",
"feature_288",
"feature_429",
"feature_253",
"feature_254",
"feature_221",
"feature_391",
"feature_256",
"feature_217",
"feature_257",
"feature_211",
"feature_304",
"feature_200_weekday",
"feature_331",
"feature_176",
"feature_200_day",
"feature_66",
"feature_152",
"feature_362",
"feature_157",
"feature_158",
"feature_16",
"feature_161",
"feature_340",
"feature_363",
"feature_61",
"feature_165",
"feature_6",
"feature_171",
"feature_172",
"feature_173",
"feature_174",
"feature_15",
"feature_144",
"feature_117",
"feature_119",
"feature_349",
"feature_130",
"feature_132",
"feature_134",
"feature_136",
"feature_137",
"feature_74",
"feature_84",
"feature_14",
"feature_85",
"feature_72",
"feature_345",
"feature_142",
"feature_70",
"feature_12",
"feature_353",
"feature_80",
"feature_351",
"feature_354",
"feature_195_year",
"feature_329",
"feature_90",
"feature_195_month",
"feature_112",
"feature_9",
"feature_332",
"feature_194_month",
"feature_105",
"feature_368",
"feature_367",
"feature_58",
"feature_103",
"feature_192_weekday",
"feature_366",
"feature_192_month",
"feature_336",
"feature_365",
"feature_191_day",
"feature_19",
"feature_338",
"feature_36",
"feature_364",
"feature_183",
"feature_20",
"feature_181",
"feature_56",
"feature_114",
"feature_179",
"feature_93"]

In [49]:
X = X[important_features]

In [50]:
#Dropping highly correlated attributes 
corr = X.corr()
# sns.heatmap(corr)
#
columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False
selected_columns = X.columns[columns]
X = X[selected_columns]

In [51]:
selected_columns

Index(['feature_2', 'feature_189', 'feature_191_year', 'feature_81',
       'feature_206', 'feature_159', 'feature_457', 'feature_352',
       'feature_17', 'feature_168',
       ...
       'feature_366', 'feature_336', 'feature_19', 'feature_338', 'feature_36',
       'feature_183', 'feature_20', 'feature_181', 'feature_114',
       'feature_179'],
      dtype='object', length=159)

In [52]:
train_df = train[selected_columns]
test_df = test[selected_columns]
train_df.shape,test_df.shape

((70000, 159), (30000, 159))

In [53]:
train_df['target'] = train['target']
train_df['id'] = train["id"]
test_df['id'] = test["id"]
train_df.shape,test_df.shape

C:\Users\DEV\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\DEV\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\DEV\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

((70000, 161), (30000, 160))

In [54]:
train_df.to_csv('trainlgbimputed2.csv',index=False)
test_df.to_csv('testlgbimputed2.csv',index=False)